# Sum all files
This script creates an overview over the data

In [12]:
import pandas as pd
import sys
sys.path.insert(0, '../../data/lib/')
import consts
from pdfexport import *
import openpyxl

In [13]:
LIST_OUT = '../../data/3. transformation/0_list_2017.csv'
ACCUM_OUT = '../../data/3. transformation/0_accumulations_2017.csv'
SUM_OUT = '../../data/3. transformation/0_overview_all.xlsx'
#exclude_list = ['novartis', 'takeda', 'menarini']
exclude_list = []

In [14]:
#Read List
df_list = pd.read_csv(LIST_OUT, names = consts.fix_columns + [consts.column_export_information], skiprows=1)
df_accumulations_simple = pd.read_csv(ACCUM_OUT, names = consts.fix_columns_accumulations, skiprows=1)


#Remove files, which we have not yet properly cleaned
for excl in exclude_list:
    df_list = df_list[df_list.source != excl]
    df_accumulations_simple = df_accumulations_simple[df_accumulations_simple.source != excl]

#Numberize
df_list = amounts_to_number(df_list)
df_accumulations_simple = amounts_to_number(df_accumulations_simple)
df_accumulations_detailed = df_accumulations_simple.copy()

#Made Simple Accumulations
df_accumulations_simple = df_accumulations_simple.replace("hcp_amount", 'hcp')
df_accumulations_simple = df_accumulations_simple.replace("hco_amount", 'hco')
df_accumulations_simple = df_accumulations_simple[~df_accumulations_simple.type.isin(['hco_count', 'hcp_count'])]

#Concat
df_concat_simple = pd.concat([df_list, df_accumulations_simple], sort=True)
df_concat_detailed = pd.concat([df_list, df_accumulations_detailed], sort=True)

#Group
df_grouped_sum = df_concat_simple.groupby(by=['source']).sum()
df_grouped_simple = df_concat_simple.groupby(by=['source', 'type']).sum()
df_grouped_detail = df_concat_detailed.groupby(by=['source', 'type']).sum()

#Group
df_grouped_sum = df_grouped_sum[['donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'total']]
df_grouped_simple = df_grouped_simple[['donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'total']]
df_grouped_detail = df_grouped_detail[['donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'total']]

In [15]:
#Save to Excel
writer = pd.ExcelWriter(SUM_OUT, options={'encoding':'utf-8'})

# Write the frame to excel
df_grouped_sum.to_excel(writer, 'total', index=True)
df_grouped_simple.to_excel(writer, 'detail_inkl_anonym', index=True)
df_grouped_detail.to_excel(writer, 'detail', index=True)

writer.save()

#import subprocess, os, sys
#subprocess.call(('open', SUM_OUT))